In [1]:
from pathlib import Path

NEEDED = set([x.split()[0] for x in """
byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
""".strip().splitlines()])
# cid (Country ID)

text = Path("input.txt").read_text()
passports = text.split("\n\n")

def split(text):
    return text.strip().split("\n\n")

def parse(passport):
    fields = passport.strip().split()
    return dict([field.split(":") for field in fields])

def keys_ok(passport):
    return NEEDED.issubset(passport.keys())
    
parse(passports[0]), keys_ok(parse(passports[0]))

({'hgt': '159cm',
  'pid': '561068005',
  'eyr': '2025',
  'iyr': '2017',
  'cid': '139',
  'ecl': 'blu',
  'hcl': '#ceb3a1',
  'byr': '1940'},
 True)

### Part 1

In [3]:
test_text = """
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
"""

assert [keys_ok(parse(p)) for p in split(test_text)] == [True, False, True, False]

In [5]:
sum(1 if keys_ok(parse(p)) else 0 for p in split(text))

202

### Part 2

```
byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
```

In [6]:
def is_valid(passport):
    try:
        assert 1920 <= int(passport["byr"]) <= 2002
        assert 2010 <= int(passport["iyr"]) <= 2020
        assert 2020 <= int(passport["eyr"]) <= 2030
        if passport["hgt"].endswith("cm"):
            assert 150 <= int(passport["hgt"].rstrip("cm")) <= 193
        elif passport["hgt"].endswith("in"):
            assert 59 <= int(passport["hgt"].rstrip("in")) <= 76
        else:
            return False
        assert passport["hcl"].startswith("#")
        assert len(passport["hcl"]) == 7
        assert set(passport["hcl"][1:]).issubset(set('abcdef0123456789'))
        assert passport["ecl"] in "amb blu brn gry grn hzl oth".split()
        assert len(passport["pid"]) == 9
        assert int(passport["pid"])
    except (AssertionError, ValueError, KeyError):
        return False
    return True

In [7]:
invalid_text = """
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""

assert not any([is_valid(parse(p)) for p in split(invalid_text)])

valid_text = """
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""

assert all([is_valid(parse(p)) for p in split(valid_text)])

In [8]:
sum(1 if is_valid(parse(p)) else 0 for p in split(text))

137